In [1]:
import sys
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, LSTM
from keras.layers.embeddings import Embedding

/home/ahmad/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
raw_text = open('wonderland.txt').read()
raw_text = raw_text.lower()
print(raw_text[:200])

alice's adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0

chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of


In [3]:
chars = sorted(list(set(raw_text)))
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters:", n_chars)
print("Total Vocab:", n_vocab)
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

Total Characters: 144413
Total Vocab: 45


In [5]:
from nltk import tokenize
sentences = tokenize.sent_tokenize(raw_text)
lengths = []
for i in range(len(sentences)):
    lengths.append(len(sentences[i]))
lengths = np.array(lengths)
sentences[0]

"alice's adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 3.0\n\nchapter i. down the rabbit-hole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into the\nbook her sister was reading, but it had no pictures or conversations in\nit, 'and what is the use of a book,' thought alice 'without pictures or\nconversations?'"

In [7]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length):
    seq_in  = raw_text[i: i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print('Total patterns:', n_patterns)

Total patterns: 144313


# Small LSTM Network : Constant Characters

In [8]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 45)                11565     
Total params: 275,757
Trainable params: 275,757
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [12]:
model.fit(X_train, Y_train, nb_epoch=20, batch_size=64, callbacks=callbacks_list)

/home/ahmad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/20
144313/144313 [==============================] - 1297s 9ms/step - loss: 2.8902
Epoch 2/20


/home/ahmad/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:432: RuntimeWarning: Can save best model only with val_categorical_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


144313/144313 [==============================] - 1215s 8ms/step - loss: 2.6727
Epoch 3/20
144313/144313 [==============================] - 877s 6ms/step - loss: 2.5711
Epoch 4/20
144313/144313 [==============================] - 930s 6ms/step - loss: 2.4910
Epoch 5/20
144313/144313 [==============================] - 895s 6ms/step - loss: 2.4160
Epoch 6/20
144313/144313 [==============================] - 878s 6ms/step - loss: 2.3500
Epoch 7/20
144313/144313 [==============================] - 1124s 8ms/step - loss: 2.2935
Epoch 8/20
144313/144313 [==============================] - 1374s 10ms/step - loss: 2.2370
Epoch 9/20
144313/144313 [==============================] - 844s 6ms/step - loss: 2.1853
Epoch 10/20
144313/144313 [==============================] - 847s 6ms/step - loss: 2.1407
Epoch 11/20
144313/144313 [==============================] - 845s 6ms/step - loss: 2.0942
Epoch 12/20
144313/144313 [==============================] - 848s 6ms/step - loss: 2.0529
Epoch 13/20
144313/144313

In [ ]:
# load the network weights
#filename = "weights-improvement-16-2.0185.hdf5"
#model.load_weights(filename)
#model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
print("Seed:")
print("\"" + ''.join([int_to_char[value] for value in pattern]) + "\"")
print("\nGenerated Sequence:")
for i in xrange(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"r voice. nobody
moved.

'who cares for you?' said alice, (she had grown to her full size by this
tim"

Generated Sequence:


NameError: name 'xrange' is not defined

# Small LSTM Network : Constant Characters With Embedding Layers

In [ ]:
X_train = np.reshape(dataX, (n_patterns, seq_length))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [ ]:
embedding_vector_length = 32
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_vector_length, input_length=seq_length))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(Y_train.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')
print model1.summary()

In [ ]:
model1.fit(X_train, Y_train, nb_epoch=10, batch_size=512)

In [ ]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
print("Seed:")
print("\"" + ''.join([int_to_char[value] for value in pattern]) + "\"")
for i in xrange(500):
    x = np.reshape(pattern, (1, len(pattern)))
    x = x/float(n_vocab)
    prediction = model1.predict(x)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

# Deep LSTM Network : Constant Characters

In [ ]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print model.summary()

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X_train, Y_train, nb_epoch=2, batch_size=256, callbacks=callbacks_list)